In [1]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from tf_lab.fundamentals.utils import set_visible_GPUs
import tf_lab.point_clouds.in_out as pio

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id
from tf_lab.point_clouds.in_out import PointCloudDataSet
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec


from tf_lab.autopredictors.evaluate.basics import eval_model, read_saved_epochs
from tf_lab.autopredictors.plotting.basics import plot_original_pclouds_vs_reconstructed,\
                                                  plot_train_val_test_curves, plot_reconstructions_at_epoch

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from geo_tool import Point_Cloud, Mesh

seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)


%matplotlib inline
%load_ext autoreload
%autoreload 2

Loading manual conv_3d_transpose.
External Losses (Chamfer-EMD) cannot be loaded.
Loading manual conv_3d_transpose.


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
from tf_lab.fundamentals.utils import reset_tf_graph

In [17]:
reset_tf_graph()

n_input = [1024, 3]

decoder_args = {'layer_sizes': [1024, np.prod(n_input)], 
                'non_linearity': tf.nn.relu, 
                'b_norm': False
               }

encoder_args = {'b_norm': False, 'spn': False, 'symmetry': tf.reduce_sum}

experiment_name = 'test'
 
conf = Conf(
            n_input = n_input,
            training_epochs = 1000,
            batch_size = 50,
            loss = 'l2',
            train_dir = '',
            loss_display_step = 1,
            saver_step = 10,
            learning_rate = 0.00002,
            saver_max_to_keep = 100,
            gauss_augment = {'mu': 0, 'sigma': 0.02},
            
            encoder = enc_dec.encoder_with_convs_and_symmetry,
            encoder_args = encoder_args,
            decoder = enc_dec.decoder_with_fc_only,
            decoder_args = decoder_args,
            experiment_name = experiment_name
           )


ae = PointNetAutoEncoder('test', conf)

Loading manual conv_3d_transpose.


In [12]:
ae.graph.get_all_collection_keys()

['layer_tensor/test/encoder_conv_layer_2',
 'layer_tensor/test/decoder_fc_0',
 'trainable_variables',
 'variables',
 'layer_tensor/test/decoder_fc_1',
 'layer_variables/test/decoder_fc_1',
 'layer_variables/test/decoder_fc_0',
 'train_op',
 'activations',
 'layer_tensor/test/encoder_conv_layer_1',
 'layer_tensor/test/encoder_conv_layer_0',
 'layer_variables/test/encoder_conv_layer_0',
 'layer_variables/test/encoder_conv_layer_1',
 'layer_variables/test/encoder_conv_layer_2']